# Obtaining SEC Financial Data

Change the names of the companies below to select the ones to be analyzed

In [25]:
import pandas as pd
import numpy as np
import datetime
import json

from earningscall import company
from secfsdstools.update import update
from secfsdstools.c_index.companyindexreading import CompanyIndexReader
from secfsdstools.c_index.searching import IndexSearch
from secfsdstools.e_collector.reportcollecting import SingleReportCollector
from secfsdstools.e_filter.rawfiltering import ReportPeriodRawFilter
from secfsdstools.e_presenter.presenting import StandardStatementPresenter

#Prevent logging message prints
import logging

from sqlalchemy import column

logging.getLogger('secfsdstools').setLevel(logging.ERROR)

#Update DB
print("Updating SEC DB...")
update()
print("---Done.")

#Get CIK for Each of Companies
companyNames = [
    "Apple Inc",
    "Johnson & Johnson",
    "JPMorgan Chase",
    "Exxon",
    "Lockheed Martin",
    "NVIDIA CORP"
]

Updating SEC DB...
---Done.


### Company Class, used to store all information associated with company

In [26]:
## Company Class: Stores information from a given CIK
class Company:
    def __init__(self, cik):
        self.cik = cik
        self.report_reader = CompanyIndexReader.get_company_index_reader(cik=self.cik)
        self.consolidated_filings = pd.DataFrame(columns=['tag'])
        self.secLinks = dict()
        self.ticker = ""
        self.transcripts = {}

    def get_cik(self):
        return self.cik

    def get_report_reader(self):
        return self.report_reader

    def getAvailableReports(self):
        return list(self.report_reader.get_all_company_reports_df()['form'].unique())

    def getFilingList(self, reportType, startDate, endDate):
        if reportType == 'All':
            unfilteredDF = self.report_reader.get_all_company_reports_df()
        else:
            unfilteredDF = self.report_reader.get_all_company_reports_df(forms=reportType)

        filteredDF = unfilteredDF[(unfilteredDF.period >= startDate) & (unfilteredDF.period <= endDate)]
        return filteredDF

    def getConsolidatedFilings(self):
        return self.consolidated_filings

    def appendFilings(self, df, filingDate):
        """
        Appends new filings data to the consolidated_filings DataFrame from a dataframe input.
        Includes both tag and stmt columns.

        Parameters:
        - df: pandas DataFrame containing 'tag', 'stmt', and 'merged' columns
        - filingDate: str representing the filing date to be used as column name
        """
        # Get list of tags from input DataFrame
        new_tags = df[['tag', 'stmt']].copy()

        # If this is the first data being added, initialize with both columns
        if len(self.consolidated_filings) == 0:
            self.consolidated_filings = pd.DataFrame(columns=['tag', 'stmt'])

        # Convert existing tag-stmt combinations to set for efficient comparison
        existing_combinations = set(zip(self.consolidated_filings['tag'], self.consolidated_filings['stmt']))
        new_combinations = set(zip(new_tags['tag'], new_tags['stmt']))
        combinations_to_add = new_combinations - existing_combinations

        # Add new tag-stmt combinations if any
        if combinations_to_add:
            new_rows = pd.DataFrame(list(combinations_to_add), columns=['tag', 'stmt'])
            self.consolidated_filings = pd.concat([self.consolidated_filings, new_rows], ignore_index=True)

        # Create the filing date column if it doesn't exist
        if filingDate not in self.consolidated_filings.columns:
            self.consolidated_filings[filingDate] = None

        # Update values for all tags in the input DataFrame
        for _, row in df.iterrows():
            mask = (self.consolidated_filings['tag'] == row['tag']) & (self.consolidated_filings['stmt'] == row['stmt'])
            # Update the value in the filing date column
            self.consolidated_filings.loc[mask, filingDate] = row['merged']

        # Replace all NaN with None
        self.consolidated_filings = self.consolidated_filings.replace({np.nan: None})

        return

    def addSecFormLink(self, period, url):
        self.secLinks[period] = url
        return

    def getSecLink(self):
        return self.secLinks

    def addTicker(self, ticker):
        self.ticker = ticker
        return

    def getTicker(self):
        return self.ticker

    def add_transcript(self, date, transcript_text):
        self.transcripts[date] = transcript_text

    def get_transcript(self, date):
       return self.transcripts.get(date)

Create Company objects for all of the companies to be analyzed

In [27]:
#Determine Company CIK from Name
companyObjDict = dict()
index_search = IndexSearch.get_index_search()
for c in companyNames:
    results = index_search.find_company_by_name(c)
    if len(results) == 1:
        print("CIK for {} : {}".format(c, results.iloc[0]['cik']))
        companyObjDict[c] = Company(cik=results.iloc[0]['cik'])
    else:
        print("-------------------------------------------------")
        print("Multiple CIK for company name {} found:".format(c))
        for index, row in results.iterrows():
            print(index, row['cik'], row['name'])
        selectedIndex = int(input("Select company index from list: "))
        companyObjDict[results.iloc[selectedIndex]['name']] = Company(cik=results.iloc[selectedIndex]['cik'])


CIK for Apple Inc : 320193
CIK for Johnson & Johnson : 200406
CIK for JPMorgan Chase : 19617
CIK for Exxon : 34088
CIK for Lockheed Martin : 936468
CIK for NVIDIA CORP : 1045810


Function used to select only the current period data from a given report

In [28]:
def select_value(row):
    # Get non-NaN values and their indices
    non_nan = [(i, val) for i, val in enumerate(row) if pd.notna(val)]
    if len(non_nan) == 1:
        return non_nan[0][1]
    elif len(non_nan) == 2:
        return non_nan[-1][1]
    return None

Obtain all 10Q/10K reports for each of the companies, we will be focusing only on the income statement as the purpose of this paper is to determine the impact of revenue drivers

In [29]:
import pandas as pd
from pathlib import Path
import datetime
import os

def get_latest_filing_date(filings_df):
    if filings_df.empty:
        return None
    return filings_df['period'].max()

def should_update_filings(csv_path, latest_filing_date):
    if not os.path.exists(csv_path):
        return True
    csv_date = str(csv_path).split('_')[-1].replace('.csv', '')
    return csv_date < latest_filing_date

def process_company_filings(companyObjDict):
    for name, obj in companyObjDict.items():
        # Get latest filings
        filingList = obj.getFilingList(
            reportType=['10-K','10-Q'],
            startDate=0,
            endDate=int(datetime.date.today().strftime('%Y%m%d'))
        ).sort_values('period', ascending=True)

        latest_filing_date = str(filingList['period'].max())
        company_name = name.replace(' ', '_')
        csv_filename = Path("filings") / f"{company_name}_consolidated_filings_{latest_filing_date}.csv"
        csv_filename.parent.mkdir(exist_ok=True)  # Add here


        # Check if we need to update
        if not should_update_filings(csv_filename, latest_filing_date):
            print(f"Using cached data for {name} from {csv_filename}")
            obj.consolidated_filings = pd.read_csv(csv_filename, index_col=0)
            continue

        print(f"Processing {filingList.shape[0]} reports for {name}...")

        for row in filingList.itertuples():
            collector = SingleReportCollector.get_report_by_adsh(adsh=row.adsh, stmt_filter=['IS'])
            rawdatabag = collector.collect()
            df = (rawdatabag.filter(ReportPeriodRawFilter())
                           .join()
                           .present(StandardStatementPresenter()))

            cols_after_inpth = df.loc[:, df.columns[df.columns.get_loc('inpth') + 1:]]
            df['merged'] = cols_after_inpth.apply(select_value, axis=1)

            filing_date = datetime.datetime.strptime(str(row.period), '%Y%m%d')
            obj.appendFilings(df, filing_date)
            obj.addSecFormLink(row.period, row.url)

        # Save to CSV with latest filing date in filename
        obj.consolidated_filings.to_csv(csv_filename, index=True)
        print(f"Saved {name} filings to {csv_filename}")


In [30]:
process_company_filings(companyObjDict)

Using cached data for Apple Inc from filings/Apple_Inc_consolidated_filings_20240630.csv
Using cached data for Johnson & Johnson from filings/Johnson_&_Johnson_consolidated_filings_20240630.csv
Using cached data for JPMorgan Chase from filings/JPMorgan_Chase_consolidated_filings_20240630.csv
Using cached data for Exxon from filings/Exxon_consolidated_filings_20240630.csv
Using cached data for Lockheed Martin from filings/Lockheed_Martin_consolidated_filings_20240630.csv
Using cached data for NVIDIA CORP from filings/NVIDIA_CORP_consolidated_filings_20240731.csv


#### Note:

Sales and Revenue numbers are somestimes not reported under the same tag use the below to investigrate.
exampleFilings = companyObjDict[companyNames[0]].getConsolidatedFilings()
exampleFilings[exampleFilings['tag'].str.contains('revenue', case=False, na=False)]

exampleFilings[exampleFilings['tag'].str.contains('sales', case=False, na=False)]

secLinks = companyObjDict[companyNames[0]].getSecLink()
secLinks[20181231]

## Printout of Income Statement for first company

In [31]:
companyObjDict[companyNames[0]].getConsolidatedFilings()

,tag,stmt,2009-06-30 00:00:00,2009-09-30 00:00:00,2009-12-31 00:00:00,2010-03-31 00:00:00,2010-06-30 00:00:00,2010-09-30 00:00:00,2010-12-31 00:00:00,2011-03-31 00:00:00,...,2022-03-31 00:00:00,2022-06-30 00:00:00,2022-09-30 00:00:00,2022-12-31 00:00:00,2023-03-31 00:00:00,2023-06-30 00:00:00,2023-09-30 00:00:00,2023-12-31 00:00:00,2024-03-31 00:00:00,2024-06-30 00:00:00
0,OperatingExpenses,IS,1.351000e+09,5.482000e+09,1.686000e+09,1.646000e+09,1.902000e+09,7.299000e+09,2.471000e+09,2.344000e+09,...,1.258000e+10,1.280900e+10,5.134500e+10,1.431600e+10,1.365800e+10,1.341500e+10,5.484700e+10,1.448200e+10,1.437100e+10,1.432600e+10
1,WeightedAverageNumberOfSharesOutstandingBasic,IS,8.937120e+08,8.930160e+08,9.035420e+08,9.075480e+08,9.121970e+08,9.094610e+08,9.192940e+08,9.231960e+08,...,1.627880e+10,1.616294e+10,1.621596e+10,1.589272e+10,1.578715e+10,1.569761e+10,1.574423e+10,1.550976e+10,1.540586e+10,1.528752e+10
2,SellingGeneralAndAdministrativeExpense,IS,1.010000e+09,4.149000e+09,1.288000e+09,1.220000e+09,1.438000e+09,5.517000e+09,1.896000e+09,1.763000e+09,...,6.193000e+09,6.012000e+09,2.509400e+10,6.607000e+09,6.201000e+09,5.973000e+09,2.493200e+10,6.786000e+09,6.468000e+09,6.320000e+09
3,IncomeTaxExpenseBenefit,IS,5.030000e+08,2.280000e+09,1.380000e+09,9.550000e+08,1.039000e+09,4.527000e+09,1.959000e+09,1.913000e+09,...,5.129000e+09,3.624000e+09,1.930000e+10,5.625000e+09,4.222000e+09,2.852000e+09,1.674100e+10,6.407000e+09,4.422000e+09,4.046000e+09
4,OperatingIncomeLoss,IS,1.672000e+09,7.658000e+09,4.725000e+09,3.979000e+09,4.234000e+09,1.838500e+10,7.827000e+09,7.874000e+09,...,2.997900e+10,2.307600e+10,1.194370e+11,3.601600e+10,2.831800e+10,2.299800e+10,1.143010e+11,4.037300e+10,2.790000e+10,2.535200e+10
5,NetIncomeLoss,IS,1.229000e+09,5.704000e+09,3.378000e+09,3.074000e+09,3.253000e+09,4.308000e+09,6.004000e+09,5.987000e+09,...,2.501000e+10,1.944200e+10,9.980300e+10,2.999800e+10,2.416000e+10,1.988100e+10,9.699500e+10,3.391600e+10,2.363600e+10,2.144800e+10
6,WeightedAverageNumberOfDilutedSharesOutstanding,IS,9.091600e+08,9.070050e+08,9.197830e+08,9.228780e+08,9.273610e+08,9.247120e+08,9.331540e+08,9.359440e+08,...,1.640332e+10,1.626220e+10,1.632582e+10,1.595572e+10,1.584705e+10,1.577502e+10,1.581255e+10,1.557664e+10,1.546471e+10,1.534818e+10
7,NonoperatingIncomeExpense,IS,6.000000e+07,3.260000e+08,3.300000e+07,5.000000e+07,5.800000e+07,1.550000e+08,1.360000e+08,2.600000e+07,...,1.600000e+08,-1.000000e+07,-3.340000e+08,-3.930000e+08,6.400000e+07,-2.650000e+08,-5.650000e+08,-5.000000e+07,1.580000e+08,1.420000e+08
8,SalesRevenueNet,IS,8.337000e+09,3.653700e+10,1.568300e+10,1.349900e+10,1.570000e+10,2.034300e+10,2.674100e+10,2.466700e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,EarningsPerShareBasic,IS,1.380000e+00,6.390000e+00,3.740000e+00,3.390000e+00,3.570000e+00,4.710000e+00,6.530000e+00,6.490000e+00,...,1.540000e+00,1.200000e+00,6.150000e+00,1.890000e+00,1.530000e+00,1.270000e+00,6.160000e+00,2.190000e+00,1.530000e+00,1.400000e+00


# Obtaining Macroeconomic indicators for years of company reports

The key OECD macroeconomic indicators that significantly impact company revenues:

1. GDP Growth Rate
- Direct impact on consumer spending and business investment
- Leading indicator of market expansion/contraction

2. Interest Rates
- Affects borrowing costs and investment decisions
- Influences consumer spending on credit-dependent purchases

3. Inflation Rate (CPI)
- Impacts purchasing power and consumer spending
- Influences pricing strategies and profit margins

4. Employment Rate
- Drives consumer confidence and spending power
- Affects labor costs and workforce availability

5. Industrial Production Index
- Indicates manufacturing sector health
- Supply chain implications

6. Consumer Confidence Index
- Predicts spending patterns
- Early indicator of demand changes

7. Trade Balance
- International market opportunities
- Exchange rate implications

8. Retail Sales
- Direct measure of consumer spending
- Sector-specific growth indicators


In [32]:
def load_oecd_data(filepath):
    """
    Load and transform OECD data into a structured dataframe with metrics as rows
    and time periods as columns.
    """
    # Read the CSV file
    df = pd.read_csv(filepath)

    # Convert TIME_PERIOD to datetime for proper sorting
    df['TIME_PERIOD'] = pd.to_datetime(df['TIME_PERIOD'].str.replace('Q', ''), format='%Y-%m') + pd.offsets.QuarterEnd()

    # Create multi-index for rows
    df_pivot = df.pivot_table(
        index=['INDICATOR', 'SUBJECT', 'MEASURE'],
        columns='TIME_PERIOD',
        values='OBS_VALUE',
        aggfunc='mean'
    )

    # Sort columns in ascending order
    df_pivot = df_pivot[sorted(df_pivot.columns)]

    # Create more readable index labels
    indicator_map = {
        'CPI': 'Consumer Price Index',
        'EMP': 'Employment',
        'INDPROD': 'Industrial Production',
        'LTINT': 'Long-term Interest Rate',
        'STINT': 'Short-term Interest Rate'
    }

    subject_map = {
        'ENRG': 'Energy',
        'FOOD': 'Food',
        'TOT': 'Total',
        'TOT_FOODENRG': 'Core (excl. Food & Energy)',
        'MEN': 'Men',
        'WOMEN': 'Women',
        'CONSTR': 'Construction',
        'MFG': 'Manufacturing'
    }

    measure_map = {
        'AGRWTH': 'Annual Growth Rate (%)',
        'IDX2015': 'Index (2015=100)',
        'PC_WKGPOP': '% of Working Age Population',
        'THND_PER': 'Thousand Persons',
        'PC_PA': '% per Annum'
    }

    df_pivot.index = pd.MultiIndex.from_tuples(
        [(indicator_map.get(i, i), subject_map.get(s, s), measure_map.get(m, m))
         for i, s, m in df_pivot.index],
        names=['Indicator', 'Subject', 'Measure']
    )

    # Sort index
    df_pivot = df_pivot.sort_index()

    return df_pivot

def format_oecd_dataframe(df):
    """
    Format the OECD dataframe by rounding numbers and handling NaN values.
    """
    formatted_df = df.copy()
    formatted_df = formatted_df.round(2)
    formatted_df = formatted_df.fillna('-')
    return formatted_df

In [33]:
# Load the data
macroIndicatorsDF = load_oecd_data('OECD,DF_DP_LIVE,+USA.STINT+LTINT+INDPROD+EMP+CPI...Q.csv')

# For better display, use the formatting function
macroIndicatorsDF = format_oecd_dataframe(macroIndicatorsDF)
macroIndicatorsDF

TIME_PERIOD                                                                      2009-03-31  \
Indicator                Subject                    Measure                                   
Consumer Price Index     Core (excl. Food & Energy) Annual Growth Rate (%)             1.69   
                                                    Index (2015=100)                  90.34   
                         Energy                     Annual Growth Rate (%)           -23.67   
                                                    Index (2015=100)                  93.75   
                         Food                       Annual Growth Rate (%)             1.60   
                                                    Index (2015=100)                  89.04   
                         Total                      Annual Growth Rate (%)            -0.94   
                                                    Index (2015=100)                  90.29   
Employment               Men                        % of Working Age Population       72.29   
                                                    Thousand Persons               73963.45   
                         Total                      % of Working Age Population       67.95   
                                                    Thousand Persons              140402.57   
                         Women                      % of Working Age Population       63.71   
                                                    Thousand Persons               66439.11   
Industrial Production    Construction               Index (2015=100)                  85.83   
                         Manufacturing              Index (2015=100)                  87.55   
                         Total                      Index (2015=100)                  85.68   
Long-term Interest Rate  Total                      % per Annum                        3.19   
Short-term Interest Rate Total                      % per Annum                        0.67   

TIME_PERIOD                                                                      2009-06-30  \
Indicator                Subject                    Measure                                   
Consumer Price Index     Core (excl. Food & Energy) Annual Growth Rate (%)             1.74   
                                                    Index (2015=100)                  90.97   
                         Energy                     Annual Growth Rate (%)             2.17   
                                                    Index (2015=100)                  99.48   
                         Food                       Annual Growth Rate (%)            -2.68   
                                                    Index (2015=100)                  88.02   
                         Total                      Annual Growth Rate (%)             1.44   
                                                    Index (2015=100)                  91.20   
Employment               Men                        % of Working Age Population       70.74   
                                                    Thousand Persons               72810.00   
                         Total                      % of Working Age Population       66.57   
                                                    Thousand Persons              138368.00   
                         Women                      % of Working Age Population       62.49   
                                                    Thousand Persons               65557.66   
Industrial Production    Construction               Index (2015=100)                  79.29   
                         Manufacturing              Index (2015=100)                  89.56   
                         Total                      Index (2015=100)                  87.17   
Long-term Interest Rate  Total                      % per Annum                        3.46   
Short-term Interest Rate Total                      % per Annum                        0.22   

TIME_PER

# Obtaining earnings call summaries

We will obtain a complete dataset of earnings calls

In [34]:
import requests

def cik_to_ticker(cik):
   json_file = 'company_tickers.json'

   # Download and save JSON if file doesn't exist
   if not os.path.exists(json_file):
       headers = {
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
       }
       url = 'https://www.sec.gov/files/company_tickers.json'
       response = requests.get(url, headers=headers)
       with open(json_file, 'w') as f:
           f.write(response.text)

   # Read from saved file
   df = pd.read_json(json_file).T
   cik = str(cik).zfill(10)

   if int(cik) in df['cik_str'].values:
       return df[df['cik_str'] == int(cik)]['ticker'].iloc[0]
   return None

In [35]:
import os
from pathlib import Path
from datetime import datetime
from llama_index.readers.earnings_call_transcript import EarningsCallTranscript
import time
from requests.exceptions import RetryError
from typing import Optional

def fetch_transcript(ticker: str, year: int, quarter: str, max_retries: int = 3, retry_delay: int = 60) -> Optional[dict]:
    """
    Fetch transcript with better error handling and retry logic.
    Returns transcript data dictionary if successful, None otherwise.
    """
    for attempt in range(max_retries):
        try:
            loader = EarningsCallTranscript(ticker=ticker, year=year, quarter=quarter)
            docs = loader.load_data()

            if not docs or not hasattr(docs, 'text'):
                print(f"No transcript data found for {ticker} {year} {quarter}")
                return None

            return {
                'date': datetime(year, ((int(quarter[1])-1)*3)+1, 1).strftime('%Y%m%d'),
                'year': year,
                'quarter': quarter,
                'text': docs.text
            }

        except RetryError:
            if attempt < max_retries - 1:
                print(f"Rate limit hit for {ticker} {year} {quarter}. Waiting {retry_delay} seconds... (Attempt {attempt+1}/{max_retries})")
                time.sleep(retry_delay)
            else:
                print(f"Failed to retrieve transcript for {ticker} {year} {quarter} after {max_retries} attempts")
                return None
        except Exception as e:
            print(f"Error processing transcript for {ticker} {year} {quarter}: {type(e).__name__}: {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(retry_delay)
            else:
                return None

    return None

def process_transcripts(companyObjDict, max_retries=3, retry_delay=60):
    """Main processing function with improved error handling"""
    for name, obj in companyObjDict.items():
        ticker = obj.getTicker() or cik_to_ticker(cik=obj.cik)
        obj.addTicker(ticker)

        transcript_dir = Path(f"transcripts/{ticker}")
        transcript_dir.mkdir(parents=True, exist_ok=True)

        # Get existing transcripts
        existing_transcripts = {f.stem.split('_')[1] for f in transcript_dir.glob("*.json")}

        for col in obj.getConsolidatedFilings().columns[2:]:
            date = datetime.strptime(str(col), '%Y-%m-%d %H:%M:%S')
            year = date.year
            quarter = f"Q{(date.month-1)//3 + 1}"
            date_str = date.strftime('%Y%m%d')

            transcript_path = transcript_dir / f"{ticker}_{date_str}.json"

            # Handle existing transcripts
            if date_str in existing_transcripts:
                print(f"Transcript already exists for {ticker} {year} {quarter}")
                try:
                    with open(transcript_path, 'r', encoding='utf-8') as f:
                        transcript_data = json.load(f)
                        obj.add_transcript(date, transcript_data['text'])
                    continue
                except Exception as e:
                    print(f"Error loading existing transcript {transcript_path}: {type(e).__name__}: {str(e)}")
                    # If we can't load existing transcript, try to fetch it again

            # Fetch new transcript
            transcript_data = fetch_transcript(ticker, year, quarter, max_retries, retry_delay)

            if transcript_data:
                try:
                    with open(transcript_path, 'w', encoding='utf-8') as f:
                        json.dump(transcript_data, f, ensure_ascii=False, indent=2)
                    obj.add_transcript(date, transcript_data['text'])
                    print(f"Saved transcript for {ticker} {year} {quarter}")
                except Exception as e:
                    print(f"Error saving transcript to {transcript_path}: {type(e).__name__}: {str(e)}")

            time.sleep(5)  # Rate limiting between requests

In [36]:
# Usage
process_transcripts(companyObjDict, max_retries=3, retry_delay=60)

Transcript already exists for AAPL 2009 Q2
Transcript already exists for AAPL 2009 Q3
Transcript already exists for AAPL 2009 Q4
Transcript already exists for AAPL 2010 Q1
Transcript already exists for AAPL 2010 Q2
Transcript already exists for AAPL 2010 Q3
Transcript already exists for AAPL 2010 Q4
Transcript already exists for AAPL 2011 Q1
Transcript already exists for AAPL 2011 Q2
Transcript already exists for AAPL 2011 Q3
Transcript already exists for AAPL 2011 Q4
Transcript already exists for AAPL 2012 Q1
Transcript already exists for AAPL 2012 Q2
Transcript already exists for AAPL 2012 Q3
Transcript already exists for AAPL 2012 Q4
Transcript already exists for AAPL 2013 Q1
Transcript already exists for AAPL 2013 Q2
Transcript already exists for AAPL 2013 Q3
Transcript already exists for AAPL 2013 Q4
Transcript already exists for AAPL 2014 Q1
Transcript already exists for AAPL 2014 Q2
Transcript already exists for AAPL 2014 Q3
Transcript already exists for AAPL 2014 Q4
Transcript 

KeyboardInterrupt: 

In [37]:
tempTranscript = companyObjDict[companyNames[0]].get_transcript(datetime.strptime('2009-06-30 00:00:00', '%Y-%m-%d %H:%M:%S'))
tempTranscript

"Executives: Nancy Paxton – Senior Director, IR and Corporate Finance Peter Oppenheimer – SVP and CFO Tim Cook – COO\nAnalysts: Richard Gardner – Citigroup Keith Bachman – Bank of Montreal Ben Reitzes – Barclays Capital Bill Fearnley – FTN Equity Capital Bill Shope – Credit Suisse Gene Munster – Piper Jaffray Toni Sacconaghi – Sanford Bernstein David Bailey – Goldman Sachs Scott Craig – Banc of America Charles Wolf – Needham & Co. Shannon Cross – Cross Research Chris Whitmore – Deutsche Bank\nOperator: Good day and welcome to this Apple Incorporated Apple Second Quarter Fiscal Year 2009 Earnings Release Conference Call. Today's call is being recorded. At this time for opening remarks and introductions, I would like to turn the call over to Nancy Paxton, Senior Director of Investor Relations. Please go ahead, ma'am.\nNancy Paxton: Thank you. Good afternoon and thanks to everyone for joining us. Speaking today is Apple's CFO, Peter Oppenheimer, and he will be joined by Apple's COO, Tim C

# Running experiments utilizing collected data

Split data into chunks to test LLM.
Last 4 quarters of Financial and Macro, Provide current transcript quarter.
Provide LLM instructions to predict the income statement measures

Determine gap between expected and predictions using evaluation metrics (Proposal)
Compare the results to average and last quarter results.

## Evaluating LLM Capability in Predicting Income Statement Items

The goal of this task is to evaluate the performance of a Large Language Model (LLM) in predicting items in the income statement based on historical data and contextual inputs. Follow these steps:

### 1. Data Preparation
For a given target year X and quarter Y:
- Gather the following data:
  - **Financial Income Statement**: Last 4 quarters of financial data.
  - **Macroeconomic Data**: Last 4 quarters of macroeconomic data.
  - **Earnings Call Transcript**: Year X Quarter Y-1 transcript.
- Ensure the data is formatted appropriately for the LLM input.

### 2. Prompt Engineering
- Design a prompt that incorporates all the above data and clearly specifies the task for the LLM.
  - Example: Include numerical and textual data with clear instructions to calculate the income statement items for Year X Quarter Y.
- Ensure the prompt is structured to minimize ambiguity and enhance model understanding.

### 3. Model Prediction
- Send the prompt to the LLM and retrieve the predictions for the income statement items for Year X Quarter Y.

### 4. Performance Evaluation
- Calculate the error between the predicted and actual values of the income statement items using the following metrics:
  - **Mean Squared Error (MSE)**:
    \[
    \text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (\hat{y}_i - y_i)^2
    \]
  - **Mean Absolute Error (MAE)**:
    \[
    \text{MAE} = \frac{1}{n} \sum_{i=1}^{n} |\hat{y}_i - y_i|
    \]
- Report the errors for each income statement item.


In [38]:
def prepare_data(company, macro_df, target_date):
   target_date = pd.to_datetime(target_date) - pd.DateOffset(months=3)

   financial_df = company.getConsolidatedFilings()
   available_dates = [col for col in financial_df.columns if col not in ['tag', 'stmt']]
   available_dates = pd.to_datetime(available_dates)
   available_dates = sorted(available_dates)

   target_idx = np.searchsorted(available_dates, target_date)
   if target_idx > 0:
       target_idx = min(target_idx, len(available_dates) - 1)

   start_idx = max(0, target_idx - 3)
   selected_dates = available_dates[start_idx:target_idx + 1]
   if len(selected_dates) < 4:
       raise ValueError("Not enough historical data available")

   date_strings = [d.strftime('%Y-%m-%d %H:%M:%S') for d in selected_dates]
   financial_features = financial_df[['tag', 'stmt'] + date_strings]

   macro_df = pd.DataFrame(macro_df)
   macro_df.columns = pd.to_datetime(macro_df.columns)
   macro_df = macro_df.T

   macro_features = pd.DataFrame()
   for date in selected_dates:
       closest_date = macro_df.index[np.abs(macro_df.index - date).argmin()]
       macro_features = pd.concat([macro_features, macro_df.loc[[closest_date]]])

   macro_features = macro_features.T

   prev_quarter_date = selected_dates[-1]
   transcript = company.get_transcript(prev_quarter_date)

   return financial_features, macro_features, transcript

We are predicting the values for the next quarter following the date_str.

In [39]:
# Example usage:
dateToPredict = '2012-03-31 00:00:00'
fin_features, macro_features, transcript = prepare_data(
    company=companyObjDict[companyNames[0]],
    macro_df=macroIndicatorsDF,
    target_date=dateToPredict)

In [40]:
fin_features

,tag,stmt,2011-03-31 00:00:00,2011-06-30 00:00:00,2011-09-30 00:00:00,2011-12-31 00:00:00
0,OperatingExpenses,IS,2.344000e+09,2.543000e+09,1.002800e+10,3.363000e+09
1,WeightedAverageNumberOfSharesOutstandingBasic,IS,9.231960e+08,9.261080e+08,9.242580e+08,9.310410e+08
2,SellingGeneralAndAdministrativeExpense,IS,1.763000e+09,1.915000e+09,7.599000e+09,2.605000e+09
3,IncomeTaxExpenseBenefit,IS,1.913000e+09,2.243000e+09,8.283000e+09,4.413000e+09
4,OperatingIncomeLoss,IS,7.874000e+09,9.379000e+09,3.379000e+10,1.734000e+10
5,NetIncomeLoss,IS,5.987000e+09,7.308000e+09,6.623000e+09,1.306400e+10
6,WeightedAverageNumberOfDilutedSharesOutstanding,IS,9.359440e+08,9.378100e+08,9.366450e+08,9.415720e+08
7,NonoperatingIncomeExpense,IS,2.600000e+07,1.720000e+08,4.150000e+08,1.370000e+08
8,SalesRevenueNet,IS,2.466700e+10,2.857100e+10,2.827000e+10,4.633300e+10
9,EarningsPerShareBasic,IS,6.490000e+00,7.890000e+00,7.130000e+00,1.403000e+01


In [41]:
macro_features

TIME_PERIOD                                                                      2011-03-31  \
Indicator                Subject                    Measure                                   
Consumer Price Index     Core (excl. Food & Energy) Annual Growth Rate (%)             1.49   
                                                    Index (2015=100)                  92.64   
                         Energy                     Annual Growth Rate (%)            16.78   
                                                    Index (2015=100)                 121.33   
                         Food                       Annual Growth Rate (%)             4.38   
                                                    Index (2015=100)                  92.85   
                         Total                      Annual Growth Rate (%)             3.11   
                                                    Index (2015=100)                  94.74   
Employment               Men                        % of Working Age Population       71.22   
                                                    Thousand Persons               74088.00   
                         Total                      % of Working Age Population       66.58   
                                                    Thousand Persons              139614.00   
                         Women                      % of Working Age Population       62.03   
                                                    Thousand Persons               65526.11   
Industrial Production    Construction               Index (2015=100)                  72.62   
                         Manufacturing              Index (2015=100)                  95.62   
                         Total                      Index (2015=100)                  93.23   
Long-term Interest Rate  Total                      % per Annum                        3.03   
Short-term Interest Rate Total                      % per Annum                        0.26   

TIME_PERIOD                                                                      2011-06-30  \
Indicator                Subject                    Measure                                   
Consumer Price Index     Core (excl. Food & Energy) Annual Growth Rate (%)             2.16   
                                                    Index (2015=100)                  93.62   
                         Energy                     Annual Growth Rate (%)            11.00   
                                                    Index (2015=100)                 116.87   
                         Food                       Annual Growth Rate (%)             6.05   
                                                    Index (2015=100)                  94.86   
                         Total                      Annual Growth Rate (%)             3.29   
                                                    Index (2015=100)                  95.40   
Employment               Men                        % of Working Age Population       71.73   
                                                    Thousand Persons               74930.00   
                         Total                      % of Working Age Population       66.78   
                                                    Thousand Persons              140698.70   
                         Women                      % of Working Age Population       61.94   
                                                    Thousand Persons               65768.66   
Industrial Production    Construction               Index (2015=100)                  75.58   
                         Manufacturing              Index (2015=100)                  97.18   
                         Total                      Index (2015=100)                  95.02   
Long-term Interest Rate  Total                      % per Annum                        2.05   
Short-term Interest Rate Total                      % per Annum                        0.42   

TIME_PER

In [42]:
transcript

"Executives: Peter Oppenheimer - Chief Financial Officer and Senior Vice President Nancy Paxton - IR Timothy D. Cook - Chief Executive Officer and Director\nAnalysts: Keith F. Bachman - BMO Capital Markets U.S. Richard Gardner - Citigroup Inc, Research Division Benjamin A. Reitzes - Barclays Capital, Research Division Kulbinder Garcha - Crédit Suisse AG, Research Division Mark A Moskowitz - JP Morgan Chase & Co, Research Division Katy Huberty - Morgan Stanley, Research Division Chris Whitmore - Deutsche Bank AG, Research Division A.M. Sacconaghi - Sanford C. Bernstein & Co., LLC., Research Division Scott D. Craig - BofA Merrill Lynch, Research Division Mike Abramsky - RBC Capital Markets, LLC, Research Division Bill C. Shope - Goldman Sachs Group Inc., Research Division Charles Eugene Munster - Piper Jaffray Companies, Research Division\nOperator: Good day, everyone, and welcome to the Apple Inc. Fourth Quarter Fiscal Year 2011 Earnings Release Conference Call. Today's call is being re

## Creating prompt for LLM

In [43]:
import numpy as np
import pandas as pd

def mse(y_true, y_pred):
    return np.mean((np.array(y_true) - np.array(y_pred)) ** 2)

def mae(y_true, y_pred):
    return np.mean(np.abs(np.array(y_true) - np.array(y_pred)))

def create_prompt(financial_features, macro_features, transcript, target_date):
    tags = financial_features['tag'].unique().tolist()

    # Format financial data
    fin_str = "Financial Data (Last 4 Quarters):\n"
    for _, row in financial_features.iterrows():
        fin_str += f"Tag: {row['tag']}, Statement: {row['stmt']}\n"
        date_cols = [col for col in financial_features.columns if col not in ['tag', 'stmt']]
        for col in date_cols:
            value = row[col]
            if isinstance(value, (int, float)) and not pd.isna(value):
                fin_str += f"{col}: {value:.2f}, "
        fin_str = fin_str.rstrip(", ") + "\n"

    # Format macro data
    macro_str = "Macroeconomic Indicators (Last 4 Quarters):\n"
    for date_idx, row in macro_features.iterrows():
        macro_str += f"Date: {date_idx}\n"
        for col in macro_features.columns:
            value = row[col]
            if isinstance(value, (int, float)) and not pd.isna(value):
                macro_str += f"{col}: {value:.2f}, "
        macro_str = macro_str.rstrip(", ") + "\n"

    prompt = f"""Task: Based on the historical data below, predict the numerical values for each financial metric for {target_date}.

Metrics to predict:
{tags}

{fin_str}

{macro_str}

Previous Quarter's Earnings Call Transcript:
{transcript}

Required Output Format:
Return a dictionary with each tag as key and predicted float value as value.
Example: {{"Tag1": 1234.56, "Tag2": 7890.12}}"""

    return prompt

def evaluate_predictions(predictions, actuals, tags=None):
    if tags is None:
        tags = list(set(predictions.keys()) & set(actuals.keys()))

    pred_values = [predictions[tag] for tag in tags if tag in predictions]
    actual_values = [actuals[tag] for tag in tags if tag in actuals]

    return {
        'MSE': mse(actual_values, pred_values),
        'MAE': mae(actual_values, pred_values)
    }

In [44]:
prompt = create_prompt(
    financial_features=fin_features,
    macro_features=macro_features,
    transcript=transcript,
    target_date=dateToPredict)

In [46]:
prompt[0:1000]

"Task: Based on the historical data below, predict the numerical values for each financial metric for 2012-03-31 00:00:00.\n\nMetrics to predict:\n['OperatingExpenses', 'WeightedAverageNumberOfSharesOutstandingBasic', 'SellingGeneralAndAdministrativeExpense', 'IncomeTaxExpenseBenefit', 'OperatingIncomeLoss', 'NetIncomeLoss', 'WeightedAverageNumberOfDilutedSharesOutstanding', 'NonoperatingIncomeExpense', 'SalesRevenueNet', 'EarningsPerShareBasic', 'GrossProfit', 'EarningsPerShareDiluted', 'CostOfGoodsAndServicesSold', 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments', 'ResearchAndDevelopmentExpense', 'CommonStockDividendsPerShareDeclared', 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest', 'Revenues', 'RevenueFromContractWithCustomerExcludingAssessedTax']\n\nFinancial Data (Last 4 Quarters):\nTag: OperatingExpenses, Statement: IS\n2011-03-31 00:00:00: 2344000000.00, 2011-06-30 00:00:0

## Sending prompt to OpenAI

# Created Function to get all data from files if necessary

In [53]:
import glob
import json
from datetime import datetime
import re

def prepare_data_from_files(company_ticker, company_name, target_date):
    """
    Prepare financial, macro, and transcript data from local files for analysis.

    Args:
        company_ticker (str): Company stock ticker symbol
        company_name (str): Full company name
        target_date (str): Target date for analysis

    Returns:
        tuple: (financial_features, macro_features, transcript)
    """
    target_date = pd.to_datetime(target_date) - pd.DateOffset(months=3)

    # Load consolidated filings
    pattern = fr'./filings/{company_name.replace(" ", "_")}_consolidated_filings_*.csv'
    matches = glob.glob(pattern)

    if matches:
        financial_df = pd.read_csv(matches[0], index_col=0)
    else:
        raise Exception(f"No matching filings found for {company_name}")

    # Process dates
    available_dates = [col for col in financial_df.columns if col not in ['tag', 'stmt']]
    available_dates = pd.to_datetime(available_dates)
    available_dates = sorted(available_dates)

    # Select relevant date range
    target_idx = np.searchsorted(available_dates, target_date)
    if target_idx > 0:
        target_idx = min(target_idx, len(available_dates) - 1)

    start_idx = max(0, target_idx - 3)
    selected_dates = available_dates[start_idx:target_idx + 1]
    if len(selected_dates) < 4:
        raise ValueError("Not enough historical data available")

    # Extract financial features
    date_strings = [d.strftime('%Y-%m-%d %H:%M:%S') for d in selected_dates]
    financial_features = financial_df[['tag', 'stmt'] + date_strings]

    # Load and process macro data
    macro_df = load_oecd_data('./macro_indicators/OECD,DF_DP_LIVE,+USA.STINT+LTINT+INDPROD+EMP+CPI...Q.csv')
    macro_df = format_oecd_dataframe(macro_df)

    macro_df = pd.DataFrame(macro_df)
    macro_df.columns = pd.to_datetime(macro_df.columns)
    macro_df = macro_df.T

    # Extract macro features
    macro_features = pd.DataFrame()
    for date in selected_dates:
        closest_date = macro_df.index[np.abs(macro_df.index - date).argmin()]
        macro_features = pd.concat([macro_features, macro_df.loc[[closest_date]]])

    macro_features = macro_features.T

    # Load transcript
    prev_quarter_date = selected_dates[-1]
    transcript_path = f'./transcripts/{company_ticker}/{company_ticker}_{prev_quarter_date.strftime("%Y%m%d")}.json'
    try:
        with open(transcript_path, 'r') as f:
            transcript = json.load(f)
    except FileNotFoundError:
        transcript = None
        print(f"Warning: No transcript found for {company_ticker} on {prev_quarter_date}")

    transcript = transcript['text']

    return financial_features, macro_features, transcript


def get_ticker(company_name):
   try:
       with open('./company_tickers.json', 'r') as file:
           company_data = json.load(file)
   except FileNotFoundError:
       return "Error: company_tickers.json not found"
   except json.JSONDecodeError:
       return "Error: Invalid JSON format"

   def normalize_name(name):
       # Remove all suffixes and punctuation at once
       normalized = re.sub(r'[.,]|\s*(Inc|Corp|Ltd|LLC|Limited|Corporation)\.?$', '', name)
       normalized = re.sub(r'\s+', ' ', normalized).strip().lower()
       normalized = re.sub(r'[^\w\s]', '', normalized)
       return normalized

   normalized_search = normalize_name(company_name)

   for company in company_data.values():
       company_title = normalize_name(company['title'])
       if company_title.startswith(normalized_search):
           return company['ticker']

   return f"No match found for company: {company_name}"

In [ ]:
# Example usage:
dateToPredict = '2012-03-31 00:00:00'
fin_features, macro_features, transcript = prepare_data_from_files(
    company_name=companyNames[0],
    company_ticker=get_ticker(companyNames[0]),
    target_date=dateToPredict)

In [54]:
fin_features

,tag,stmt,2011-03-31 00:00:00,2011-06-30 00:00:00,2011-09-30 00:00:00,2011-12-31 00:00:00
0,OperatingExpenses,IS,2.344000e+09,2.543000e+09,1.002800e+10,3.363000e+09
1,WeightedAverageNumberOfSharesOutstandingBasic,IS,9.231960e+08,9.261080e+08,9.242580e+08,9.310410e+08
2,SellingGeneralAndAdministrativeExpense,IS,1.763000e+09,1.915000e+09,7.599000e+09,2.605000e+09
3,IncomeTaxExpenseBenefit,IS,1.913000e+09,2.243000e+09,8.283000e+09,4.413000e+09
4,OperatingIncomeLoss,IS,7.874000e+09,9.379000e+09,3.379000e+10,1.734000e+10
5,NetIncomeLoss,IS,5.987000e+09,7.308000e+09,6.623000e+09,1.306400e+10
6,WeightedAverageNumberOfDilutedSharesOutstanding,IS,9.359440e+08,9.378100e+08,9.366450e+08,9.415720e+08
7,NonoperatingIncomeExpense,IS,2.600000e+07,1.720000e+08,4.150000e+08,1.370000e+08
8,SalesRevenueNet,IS,2.466700e+10,2.857100e+10,2.827000e+10,4.633300e+10
9,EarningsPerShareBasic,IS,6.490000e+00,7.890000e+00,7.130000e+00,1.403000e+01


In [55]:
macro_features

TIME_PERIOD                                                                      2011-03-31  \
Indicator                Subject                    Measure                                   
Consumer Price Index     Core (excl. Food & Energy) Annual Growth Rate (%)             1.49   
                                                    Index (2015=100)                  92.64   
                         Energy                     Annual Growth Rate (%)            16.78   
                                                    Index (2015=100)                 121.33   
                         Food                       Annual Growth Rate (%)             4.38   
                                                    Index (2015=100)                  92.85   
                         Total                      Annual Growth Rate (%)             3.11   
                                                    Index (2015=100)                  94.74   
Employment               Men                        % of Working Age Population       71.22   
                                                    Thousand Persons               74088.00   
                         Total                      % of Working Age Population       66.58   
                                                    Thousand Persons              139614.00   
                         Women                      % of Working Age Population       62.03   
                                                    Thousand Persons               65526.11   
Industrial Production    Construction               Index (2015=100)                  72.62   
                         Manufacturing              Index (2015=100)                  95.62   
                         Total                      Index (2015=100)                  93.23   
Long-term Interest Rate  Total                      % per Annum                        3.03   
Short-term Interest Rate Total                      % per Annum                        0.26   

TIME_PERIOD                                                                      2011-06-30  \
Indicator                Subject                    Measure                                   
Consumer Price Index     Core (excl. Food & Energy) Annual Growth Rate (%)             2.16   
                                                    Index (2015=100)                  93.62   
                         Energy                     Annual Growth Rate (%)            11.00   
                                                    Index (2015=100)                 116.87   
                         Food                       Annual Growth Rate (%)             6.05   
                                                    Index (2015=100)                  94.86   
                         Total                      Annual Growth Rate (%)             3.29   
                                                    Index (2015=100)                  95.40   
Employment               Men                        % of Working Age Population       71.73   
                                                    Thousand Persons               74930.00   
                         Total                      % of Working Age Population       66.78   
                                                    Thousand Persons              140698.70   
                         Women                      % of Working Age Population       61.94   
                                                    Thousand Persons               65768.66   
Industrial Production    Construction               Index (2015=100)                  75.58   
                         Manufacturing              Index (2015=100)                  97.18   
                         Total                      Index (2015=100)                  95.02   
Long-term Interest Rate  Total                      % per Annum                        2.05   
Short-term Interest Rate Total                      % per Annum                        0.42   

TIME_PER

In [56]:
transcript

"Executives: Peter Oppenheimer - Chief Financial Officer and Senior Vice President Nancy Paxton - IR Timothy D. Cook - Chief Executive Officer and Director\nAnalysts: Keith F. Bachman - BMO Capital Markets U.S. Richard Gardner - Citigroup Inc, Research Division Benjamin A. Reitzes - Barclays Capital, Research Division Kulbinder Garcha - Crédit Suisse AG, Research Division Mark A Moskowitz - JP Morgan Chase & Co, Research Division Katy Huberty - Morgan Stanley, Research Division Chris Whitmore - Deutsche Bank AG, Research Division A.M. Sacconaghi - Sanford C. Bernstein & Co., LLC., Research Division Scott D. Craig - BofA Merrill Lynch, Research Division Mike Abramsky - RBC Capital Markets, LLC, Research Division Bill C. Shope - Goldman Sachs Group Inc., Research Division Charles Eugene Munster - Piper Jaffray Companies, Research Division\nOperator: Good day, everyone, and welcome to the Apple Inc. Fourth Quarter Fiscal Year 2011 Earnings Release Conference Call. Today's call is being re

In [57]:
prompt = create_prompt(
    financial_features=fin_features,
    macro_features=macro_features,
    transcript=transcript,
    target_date=dateToPredict)

In [58]:
prompt[0:1000]

"Task: Based on the historical data below, predict the numerical values for each financial metric for 2012-03-31 00:00:00.\n\nMetrics to predict:\n['OperatingExpenses', 'WeightedAverageNumberOfSharesOutstandingBasic', 'SellingGeneralAndAdministrativeExpense', 'IncomeTaxExpenseBenefit', 'OperatingIncomeLoss', 'NetIncomeLoss', 'WeightedAverageNumberOfDilutedSharesOutstanding', 'NonoperatingIncomeExpense', 'SalesRevenueNet', 'EarningsPerShareBasic', 'GrossProfit', 'EarningsPerShareDiluted', 'CostOfGoodsAndServicesSold', 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments', 'ResearchAndDevelopmentExpense', 'CommonStockDividendsPerShareDeclared', 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest', 'Revenues', 'RevenueFromContractWithCustomerExcludingAssessedTax']\n\nFinancial Data (Last 4 Quarters):\nTag: OperatingExpenses, Statement: IS\n2011-03-31 00:00:00: 2344000000.00, 2011-06-30 00:00:0